<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#地铁和商圈" data-toc-modified-id="地铁和商圈-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>地铁和商圈</a></span></li><li><span><a href="#数据生成" data-toc-modified-id="数据生成-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>数据生成</a></span><ul class="toc-item"><li><span><a href="#新房" data-toc-modified-id="新房-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>新房</a></span></li><li><span><a href="#二手房" data-toc-modified-id="二手房-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>二手房</a></span></li><li><span><a href="#租房" data-toc-modified-id="租房-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>租房</a></span></li></ul></li></ul></div>

## 地铁和商圈

In [1]:
import json
import pandas as pd
import csv

In [2]:
with open("data/bj_city_info.json") as f:
    city_data = f.read()
city_json = json.loads(city_data)

In [3]:
subway_station_cols = ['city_id',
 'city_name',
 'city_abbr',
 'subway_line_id',
 'subway_line_name',
 'baidu_subway_line_id',
 'subway_line_longitude',
 'subway_line_latitude',
 'subway_station_id',
 'subway_station_name',
 'subway_station_longitude',
 'subway_station_latitude']

In [4]:
f = open('data/subway_station.csv','w',encoding='utf-8')
csv_writer = csv.writer(f)
csv_writer.writerow(subway_station_cols)

dct = {}
dct['city_id'] = city_json['city_id']
dct['city_name'] = city_json['city_name']
dct['city_abbr'] = city_json['city_abbr']
for line in city_json["subway_line"]:
    dct['subway_line_id'] = line['subway_line_id']
    dct['subway_line_name'] = line['subway_line_name']
    dct['baidu_subway_line_id'] = line['baidu_subway_line_id']
    dct['subway_line_longitude'] = line['longitude']
    dct['subway_line_latitude'] = line['latitude']
    for station in line['station']:
        dct['subway_station_id'] = station['subway_station_id']
        dct['subway_station_name'] = station['subway_station_name']
        dct['subway_station_longitude'] = station['longitude']
        dct['subway_station_latitude'] = station['latitude']
        csv_writer.writerow(dct.values())

f.close()

In [5]:
district_info_cols = ['city_id',
 'city_name',
 'city_abbr',
 'district_id',
 'district_quanpin',
 'district_name',
 'bizcircle_id',
 'bizcircle_quanpin',
 'bizcircle_name',
 ]

In [6]:
f = open('data/district_info.csv','w',encoding='utf-8')
csv_writer = csv.writer(f)
csv_writer.writerow(district_info_cols)

dct = {}
dct['city_id'] = city_json['city_id']
dct['city_name'] = city_json['city_name']
dct['city_abbr'] = city_json['city_abbr']
for line in city_json["district"]:
    dct['district_id'] = line['district_id']
    dct['district_quanpin'] = line['district_quanpin']
    dct['district_name'] = line['district_name']
    for station in line['bizcircle']:
        dct['bizcircle_id'] = station['bizcircle_id']
        dct['bizcircle_quanpin'] = station['bizcircle_quanpin']
        dct['bizcircle_name'] = station['bizcircle_name']
        csv_writer.writerow(dct.values())

f.close()

## 数据生成

In [7]:
def dict_generator(indict, pre=None):
    pre = pre[:] if pre else []
    if isinstance(indict, dict):
        for key, value in indict.items():
            if isinstance(value, dict):
                if len(value) == 0:
                    yield pre+[key, '{}']
                else:
                    for d in dict_generator(value, pre + [key]):
                        yield d
            elif isinstance(value, list):
                if len(value) == 0:
                    yield pre+[key, '[]']
                else:
                    for v in value:
                        for d in dict_generator(v, pre + [key]):
                            yield d
            elif isinstance(value, tuple):
                if len(value) == 0:
                    yield pre+[key, '()']
                else:
                    for v in value:
                        for d in dict_generator(v, pre + [key]):
                            yield d
            else:
                yield pre + [key, value]
    else:
        yield indict

### 新房

In [8]:
import json
import pandas as pd
import copy
with open("data/bj_newhouse.json") as f:
    data_newhouse = f.read()
newhouse = json.loads(data_newhouse)

In [12]:
a = []
for index in newhouse:
    dct = {}
    for i in dict_generator(newhouse[index]):
        dct['.'.join(i[0:-1])] = i[-1]
    a.append(copy.deepcopy(dct))

In [13]:
df_newhouse = pd.DataFrame(a)

In [14]:
df_newhouse.head()

,item_type,item_type_desc,pid,id,city_id,city_name,cover_pic,min_frame_area,max_frame_area,district_name,...,北.京.江.南.投.资.集.团.有.限.公,北.京.首.城.置.业.有.限.公,避,北.京.新.博.城.房.地.产.开.发.有.限.公,北.京.日.月.房.地.产.开.发.有.限.公,北.京.天.润.福.源.房.地.产.开.发.有.限.公,北.京.牛.栏.山.房.地.产.开.发.有.限.责.任.公,北.京.青.龙.湖.盛.通.房.地.产.开.发.有.限.公,北.京.万.龙.华.开.房.地.产.开.发.有.限.公,北.京.万.越.辉.置.业.有.限.公
0,0,,656709,1120053618049262,110000,北京市,https://ke-image.ljcdn.com/newhouse-user-image...,98,195,海淀,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,,651918,1120050701448279,110000,北京市,https://ke-image.ljcdn.com/newhouse-user-image...,75,89,房山,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,,647050,1120047698911651,110000,北京市,https://ke-image.ljcdn.com/newhouse-user-image...,50,118,顺义,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,,644651,1120045667526904,110000,北京市,https://ke-image.ljcdn.com/newhouse-user-image...,75,113,房山,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,二手房,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_newhouse.to_csv("data/bj_newhouse.csv", encoding = 'utf-8')

### 二手房

In [16]:
import json
import pandas as pd
import copy
with open("data/bj_ershoufang.json") as f:
    data_ershoufang = f.read()
esf = json.loads(data_ershoufang)

In [17]:
a = []
for index in esf:
    dct = {}
    for i in dict_generator(esf[index]):
        dct['.'.join(i[0:-1])] = i[-1]
    a.append(copy.deepcopy(dct))

In [18]:
df_esf = pd.DataFrame(a)

In [19]:
df_esf.head()

,cardType,cityId,houseCode,resblockId,delegateId,title,desc,bangdanTitle,recoDesc,totalPrice,...,poiDistance,poiIcon,brandTags,marketBooth.type,marketBooth.title,marketBooth.iconUrl,marketBooth.textColor,marketBooth.bgColor,colorTags,framePicture
0,CARD_TYPE_ERSHOUFANG_LIST,110000,101113410571,1111027380887,101113410571,安定门外西河沿小区，西南两居，无遮挡，满五年,2室1厅/69.85m²/西南/安外西河沿,2室1厅/69.85m²/安外西河沿,东城 安定门/中楼层 共12层/西南,805万,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CARD_TYPE_ERSHOUFANG_LIST,110000,101113348865,1111027377922,101113348865,东城二环内两居室，商品房满五唯一,2室1厅/55.55m²/南 北/朗家胡同13号院,2室1厅/55.55m²/朗家胡同13号院,东城 安定门/低楼层 共6层/南 北,699万,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CARD_TYPE_ERSHOUFANG_LIST,110000,101113209556,1111027381347,101113209556,北二环 2006年社区 房子精装修 满两年,2室1厅/81.74m²/南/西营房,2室1厅/81.74m²/西营房,东城 安定门/低楼层 共17层/南,910万,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CARD_TYPE_ERSHOUFANG_LIST,110000,101113134769,1111027381843,101113134769,安交区域 工信部宿舍 南向一居 楼层好 随时签约,1室1厅/36.58m²/南/永康胡同5号院,1室1厅/36.58m²/永康胡同5号院,东城 安定门/中楼层 共5层/南,585万,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CARD_TYPE_ERSHOUFANG_LIST,110000,101113083390,1111027375670,101113083390,北二环内 安馨园 板楼带电梯 南北通透,2室2厅/139.78m²/南 北/安馨园,2室2厅/139.78m²/安馨园,东城 安定门/低楼层 共6层/南 北,1700万,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_esf.to_csv("data/bj_ershoufang.csv", encoding = 'utf-8')

### 租房

In [21]:
import json
import pandas as pd
import copy
with open("data/bj_zufang.json") as f:
    city_data = f.read()
zufang = json.loads(city_data)

In [22]:
a = []
for index in zufang:
    dct = {}
    for i in dict_generator(zufang[index]):
        dct['.'.join(i[0:-1])] = i[-1]
    a.append(copy.deepcopy(dct))

In [23]:
df_zufang = pd.DataFrame(a)

In [24]:
df_zufang.head()

,member_level,ad_desc,ad_code,ad_type,position,rent_type,city_id,rent_area,house_attention_type,house_code,...,1.1.0,4.5,4.6,4.7,item_type,1.1.5,1.1.3,1.1.6,1.1.7,1.1.8
0,0,,0,0,-1,200600000001,110000,60.30,,BJ2904173747973914624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,广告,47372202815669344,40,0,200600000002,110000,9.77,,BJ2250106586188496896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,,0,0,-1,200600000001,110000,162.61,,BJ2906656728277843968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,广告,47372202815669344,40,2,200600000001,110000,54.73,,BJ2363932583617626112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,,0,0,-1,200600000001,110000,64.37,,BJ1566756939084333056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_zufang.to_csv("data/bj_zufang.csv", encoding = 'utf-8')